In [ ]:

import pandas as pd
import re
from pathlib import Path
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

SRC_File='/content/drive/MyDrive/Project - Neo4j + ML/obs_df.csv'

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(SRC_File, low_memory=False)

def coalesce(a, b):
    """Prefer a; if blank/NA use b."""
    a = a.fillna("").astype(str).str.strip().replace({"": np.nan})
    b = b.fillna("").astype(str).str.strip().replace({"": np.nan})
    return a.combine_first(b)

def clean_text(s):
    """Trim whitespace; turn '', 'nan', 'None' into NA; keep original case."""
    s = s.astype(str).str.strip()
    s = s.replace({"": np.nan, "nan": np.nan, "None": np.nan, "NONE": np.nan})
    return s

In [ ]:
# Coalesced scientific names per side
pred_sci = coalesce(df.get("predator_scientific_name"), df.get("predator_taxon_species_name"))
prey_sci = coalesce(df.get("prey_scientific_name"), df.get("prey_taxon_species_name"))

In [ ]:
# Assemble candidate taxa rows from predator side
pred_taxa = pd.DataFrame({
    "scientific_name": pred_sci,
    "common_name": df.get("predator_common_name"),
    "iconic_taxon_name": df.get("predator_iconic_taxon_name"),
    "taxon_kingdom": df.get("predator_taxon_kingdom_name"),
    "taxon_id": df.get("predator_taxon_id"),
})

# …and prey side
prey_taxa = pd.DataFrame({
    "scientific_name": prey_sci,
    "common_name": df.get("prey_common_name"),
    "iconic_taxon_name": df.get("prey_iconic_taxon_name"),
    "taxon_kingdom": df.get("prey_taxon_kingdom_name"),
    "taxon_id": df.get("prey_taxon_id"),
})

In [ ]:
taxa = pd.concat([pred_taxa, prey_taxa], ignore_index=True)
taxa["scientific_name"] = clean_text(taxa["scientific_name"])
taxa = taxa.dropna(subset=["scientific_name"])

In [ ]:
def first_nonnull(series):
    return series.dropna().iloc[0] if series.notna().any() else np.nan

taxa = (taxa
        .sort_values("scientific_name")
        .groupby("scientific_name", as_index=False)
        .agg({
            "common_name": first_nonnull,
            "iconic_taxon_name": first_nonnull,
            "taxon_kingdom": first_nonnull,
            "taxon_id": first_nonnull
        }))

taxa.head(), taxa.shape

(         scientific_name                 common_name iconic_taxon_name  \
 0      Abantis paradisea            Paradise Skipper           Insecta   
 1  Abisares viridipennis  Notched Shield Grasshopper           Insecta   
 2          Abramis brama                Common Bream    Actinopterygii   
 3       Abrus laevigatus                  Lucky Bean           Plantae   
 4      Abrus precatorius                  rosary pea           Plantae   
 
   taxon_kingdom   taxon_id  
 0      Animalia   366899.0  
 1      Animalia  1642428.0  
 2      Animalia    92984.0  
 3       Plantae   578899.0  
 4       Plantae   122099.0  ,
 (3320, 5))

In [ ]:
rels = pd.DataFrame({
    "uuid": df.get("uuid"),
    "predator_scientific_name": pred_sci,
    "prey_scientific_name": prey_sci,
    "observed_on": df.get("observed_on"),
    "time_observed_at": df.get("time_observed_at"),
    "place": df.get("place_guess"),
    "place_town": df.get("place_town_name"),
    "place_county": df.get("place_county_name"),
    "place_state": df.get("place_state_name"),
    "place_country": df.get("place_country_name"),
    "latitude": df.get("latitude"),
    "longitude": df.get("longitude"),
    "special_feeding": df.get("special_type_of_feeding"),
    "url": df.get("url"),
    "image_url": df.get("image_url"),
    "predator_quality": df.get("predator_quality_grade"),
    "prey_quality": df.get("prey_quality_grade"),
})


In [ ]:
# Clean endpoints
rels["predator_scientific_name"] = clean_text(rels["predator_scientific_name"])
rels["prey_scientific_name"] = clean_text(rels["prey_scientific_name"])

# Drop rows missing either endpoint
rels = rels.dropna(subset=["predator_scientific_name", "prey_scientific_name"])


In [ ]:
# Ensure numeric types (non-numeric coerced to NaN)
rels["latitude"] = pd.to_numeric(rels["latitude"], errors="coerce")
rels["longitude"] = pd.to_numeric(rels["longitude"], errors="coerce")

In [ ]:
# 1) Ensure every relationship endpoint exists in taxa
pred_missing = set(rels["predator_scientific_name"]) - set(taxa["scientific_name"])
prey_missing = set(rels["prey_scientific_name"]) - set(taxa["scientific_name"])

In [ ]:

print("Predators missing in taxa:", len(pred_missing))
print("Prey missing in taxa:", len(prey_missing))

# 2) Basic sanity counts
print("Taxa rows:", len(taxa))
print("EATS rows:", len(rels))

Predators missing in taxa: 0
Prey missing in taxa: 0
Taxa rows: 3320
EATS rows: 5033


In [ ]:
# 3) Check for obvious duplication in scientific names (post-normalization)
dup_names = taxa["scientific_name"].value_counts()
dup_names = dup_names[dup_names > 1]
print("Duplicate scientific names:", len(dup_names))

Duplicate scientific names: 0


In [ ]:
taxa.to_csv("taxa.csv", index=False)
rels.to_csv("eats.csv", index=False)

from google.colab import files
files.download("taxa.csv")
files.download("eats.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>